In [1]:
import tweepy
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Twitter API anahtarlarınızı buraya girin
consumer_key = "YOUR_CONSUMER_KEY"
consumer_secret = "YOUR_CONSUMER_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_TOKEN_SECRET"

In [2]:
# Twitter API'ye bağlanmak
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth) 

In [3]:
# Bitcoin hakkında tweetleri çekmek
tweets = api.search(q="#Bitcoin", count=1000) 

In [4]:
# Tweet metinlerini ve tarihlerini saklamak için bir dataframe oluşturmak
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
data['dates'] = [tweet.created_at for tweet in tweets] 

In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [6]:
# Duygu analizi için NLTK kullanmak
sia = SentimentIntensityAnalyzer() 

In [7]:
# Her tweet için duygu değerlerini hesaplamak
list_of_scores = []
for index, row in data.iterrows():
    score = sia.polarity_scores(row["tweets"])
    list_of_scores.append(score) 

In [8]:
# Duygu değerlerini dataframe'e eklemek
se = pd.Series(list_of_scores)
data['polarity'] = se.values 

In [9]:
dataNew = pd.DataFrame(list(data['polarity']), columns=['neg', 'neu','pos','compound'])
dataNew

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.000,1.000,0.000,0.0000
2,0.120,0.719,0.161,0.0516
3,0.000,0.838,0.162,0.4019
4,0.000,0.731,0.269,0.7096
...,...,...,...,...
88,0.000,1.000,0.000,0.0000
89,0.000,0.799,0.201,0.7003
90,0.187,0.767,0.047,-0.6369
91,0.080,0.759,0.160,0.4404


In [10]:
dataNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   neg       93 non-null     float64
 1   neu       93 non-null     float64
 2   pos       93 non-null     float64
 3   compound  93 non-null     float64
dtypes: float64(4)
memory usage: 3.0 KB


In [11]:
dataNew['rolling_neg_std'] = dataNew['neg'].rolling(window=62).std

In [12]:
print(dataNew.head())

    neg    neu    pos  compound  \
0  0.00  1.000  0.000    0.0000   
1  0.00  1.000  0.000    0.0000   
2  0.12  0.719  0.161    0.0516   
3  0.00  0.838  0.162    0.4019   
4  0.00  0.731  0.269    0.7096   

                                     rolling_neg_std  
0  <bound method Rolling.std of Rolling [window=6...  
1  <bound method Rolling.std of Rolling [window=6...  
2  <bound method Rolling.std of Rolling [window=6...  
3  <bound method Rolling.std of Rolling [window=6...  
4  <bound method Rolling.std of Rolling [window=6...  


In [13]:
# Volatilite hesaplamak (252 günlük standart sapması)
data['rolling_std'] = data['polarity'].rolling(window=252).std

In [14]:
# Sonuçları göstermek
print(data.head())

                                              tweets               dates  \
0  RT @natbrunell: Over time:\n\n📉 Fiat\n📉 Eggs\n... 2023-01-29 22:36:45   
1  RT @paulewaulpaul: Still in the zone.\n\nThe p... 2023-01-29 22:36:45   
2  RT @saylor: #Bitcoin is a swarm of cyber horne... 2023-01-29 22:36:44   
3  RT @saylor: "Be the change you wish to see in ... 2023-01-29 22:36:44   
4  RT @saylor: While markets shudder and pundits ... 2023-01-29 22:36:44   

                                            polarity  \
0  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...   
1  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...   
2  {'neg': 0.12, 'neu': 0.719, 'pos': 0.161, 'com...   
3  {'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'comp...   
4  {'neg': 0.0, 'neu': 0.731, 'pos': 0.269, 'comp...   

                                         rolling_std  
0  <bound method Rolling.std of Rolling [window=2...  
1  <bound method Rolling.std of Rolling [window=2...  
2  <bound method Rolling.std of Rolling [

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   tweets       93 non-null     object        
 1   dates        93 non-null     datetime64[ns]
 2   polarity     93 non-null     object        
 3   rolling_std  93 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ KB
